# Face Recognition - Generate Eigenface Classifiers

This notebook tests face detection and recognition from sample images

In [1]:
import cv2, cv2.face
import os, os.path
import numpy as np
from PIL import Image

In [2]:
# constants
CASCADE_PATH = './data/'
CASCADE_NAME = 'haarcascade_frontalface_default.xml'
SAMPLE_TRAINING = './data/sample_training/'
SAMPLE_TESTING = './data/sample_testing/'

In [3]:
#vSAMPLE_TRAINING = '/home/rico/datasets/lfw/raw/'

In [4]:
# Viola-Jones classifier for Haar feature extraction
cascader_file = os.path.join(CASCADE_PATH, CASCADE_NAME)
cascader = cv2.CascadeClassifier(cascader_file)
assert(not cascader.empty())

cascader_kwargs = { # setup detection args here
    'scaleFactor': 1.1,
    'minNeighbors': 2,
    'flags': 2
}

In [5]:
from util import get_images_and_labels

In [6]:
# setup training set
training_set = get_images_and_labels(SAMPLE_TRAINING,
                                    cascader=None,
                                    debug=True,
                                    debug_accuracy=False)

Hosni_Mubarak
	Hosni_Mubarak_0007.png [1 face]
	Hosni_Mubarak_0003.png [1 face]
	Hosni_Mubarak_0004.png [1 face]
	Hosni_Mubarak_0008.png [1 face]
	Hosni_Mubarak_0005.png [1 face]
	Hosni_Mubarak_0001.png [1 face]
	Hosni_Mubarak_0006.png [1 face]
	Hosni_Mubarak_0002.png [1 face]
[Added 8 images]
Jeong_Se-hyun
	Jeong_Se-hyun_0007.png [1 face]
	Jeong_Se-hyun_0001.png [1 face]
	Jeong_Se-hyun_0008.png [1 face]
	Jeong_Se-hyun_0003.png [1 face]
	Jeong_Se-hyun_0004.png [1 face]
	Jeong_Se-hyun_0002.png [1 face]
	Jeong_Se-hyun_0005.png [1 face]
	Jeong_Se-hyun_0006.png [1 face]
[Added 16 images]
Aaron_Peirsol
	Aaron_Peirsol_0003.png [1 face]
	Aaron_Peirsol_0001.png [1 face]
	Aaron_Peirsol_0002.png [1 face]
[Added 19 images]
Heizo_Takenaka
	Heizo_Takenaka_0002.png [1 face]
	Heizo_Takenaka_0007.png [1 face]
	Heizo_Takenaka_0001.png [1 face]
	Heizo_Takenaka_0006.png [1 face]
	Heizo_Takenaka_0004.png [1 face]
	Heizo_Takenaka_0008.png [1 face]
	Heizo_Takenaka_0003.png [1 face]
	Heizo_Takenaka_0005.png 

In [7]:
# setup testing set
testing_set_cascade = get_images_and_labels(SAMPLE_TESTING,
                                    cascader=cascader,
                                    debug=False,
                                    debug_faces=True,
                                    debug_accuracy=True)

testing_set= get_images_and_labels(SAMPLE_TESTING,
                                    cascader=None,
                                    debug=False,
                                    debug_faces=True,
                                    debug_accuracy=True)


Accuracy: 100.00000%
Accuracy: 100.00000%


In [8]:
# setup labels
label_dict = {}
label_dict_inv = {}

for label in training_set[1]:
    if label not in label_dict:
        label_dict_inv[len(label_dict)] = label
        label_dict[label] = len(label_dict)
        

In [9]:
def perform_training(recognizer):
    images, raw_labels = training_set
    labels = [label_dict[label] for label in raw_labels]
    print(labels)
    recognizer.train(images, np.array(labels))
    print('Trained %d images\n' % len(images))

In [10]:
def perform_testing(recognizer, testing_set=testing_set):
    
    images, labels = testing_set
    
    print('Performing %d tests\n' % len(images))

    correct = 0
    total = 0

    for image, expected in zip(images, labels):
        
        if expected not in label_dict:
            print('No face found for %s' % expected)
            
        else:
            eid = label_dict[expected]
            lid, confidence = recognizer.predict(image)
            
            if lid in label_dict_inv:
                
                actual = label_dict_inv[lid]
                                        
                if eid == lid:
                    print('%s is correctly recognized with confidence %.10f' % (expected, confidence))
                    correct += 1
                                        
                else:
                    print('%s incorrect (recognized as %s with confidence %.10f)' % (expected, actual, confidence))
                    
            else:
                print('No face found for %s' % expected)
                
        total += 1

    print()
    print('Accuracy: %.2f%%' % (100 * (correct / max(1, total))))
    print('Correct:  %d' % correct)
    print('Wrong:    %d' % (total - correct))

In [11]:
reco = cv2.face.createLBPHFaceRecognizer()
perform_training(reco)
ims, ls = testing_set
cor = 0
tot = 0
for im, l in zip(ims,ls):
    label = label_dict[l]
    inf, conf = reco.predict(im)
    print(label, inf, conf)
    if (label == inf):
        cor += 1
    tot += 1
print(100*cor/tot,'% Accuracy')

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

0 0 70.33203017168863
1 4 66.9695790300998
2 8 64.02631180994958
3 3 62.252336352937334
4 5 55.98307131663556
5 5 58.443145513715066
6 6 58.194282662587064
7 1 61.83141922223049
8 8 60.659852760896726
9 9 69.14819846258747
60.0 % Accuracy


# 1. EigenFaces

In [12]:
# face recognizer
recognizer = cv2.face.createEigenFaceRecognizer()

In [13]:
%%time
# Perform the training
perform_training(recognizer)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

CPU times: user 532 ms, sys: 48 ms, total: 580 ms
Wall time: 579 ms


In [14]:
perform_testing(recognizer)

Performing 10 tests

Hosni_Mubarak incorrect (recognized as Jeong_Se-hyun with confidence 7708.8797264715)
Jeong_Se-hyun incorrect (recognized as Jesse_Jackson with confidence 9191.3998485021)
Aaron_Peirsol incorrect (recognized as Jeong_Se-hyun with confidence 7686.4538475870)
Heizo_Takenaka is correctly recognized with confidence 7592.1110518190
Jesse_Jackson incorrect (recognized as George_Clooney with confidence 7535.3358054781)
Hugh_Grant is correctly recognized with confidence 7144.1526959103
Fernando_Gonzalez is correctly recognized with confidence 7184.3745207685
George_Clooney is correctly recognized with confidence 8396.7570350231
Colin_Farrell incorrect (recognized as Hugh_Grant with confidence 7856.0370864173)
Charles_Taylor incorrect (recognized as Fernando_Gonzalez with confidence 9496.5626098884)

Accuracy: 40.00%
Correct:  4
Wrong:    6


# 2. FisherFaces

In [15]:
# face recognizer
recognizer = cv2.face.createFisherFaceRecognizer()

In [16]:
%%time
perform_training(recognizer)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

CPU times: user 492 ms, sys: 16 ms, total: 508 ms
Wall time: 506 ms


In [17]:
%%time
perform_testing(recognizer)

Performing 10 tests

Hosni_Mubarak incorrect (recognized as George_Clooney with confidence 1415.5985222224)
Jeong_Se-hyun incorrect (recognized as Colin_Farrell with confidence 1941.4372785408)
Aaron_Peirsol incorrect (recognized as Hosni_Mubarak with confidence 1495.4150848578)
Heizo_Takenaka incorrect (recognized as Colin_Farrell with confidence 2170.6659331031)
Jesse_Jackson incorrect (recognized as Hugh_Grant with confidence 1476.5334439179)
Hugh_Grant is correctly recognized with confidence 1683.8350626277
Fernando_Gonzalez is correctly recognized with confidence 1736.9916732560
George_Clooney is correctly recognized with confidence 1638.8801908796
Colin_Farrell incorrect (recognized as Fernando_Gonzalez with confidence 1593.3159287082)
Charles_Taylor incorrect (recognized as Hosni_Mubarak with confidence 1666.5036180065)

Accuracy: 30.00%
Correct:  3
Wrong:    7
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 5.25 ms


# 2. Local Binary Patterns

In [18]:
# face recognizer
recognizer = cv2.face.createLBPHFaceRecognizer()

In [19]:
%%time
perform_training(recognizer)

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
Trained 75 images

CPU times: user 204 ms, sys: 0 ns, total: 204 ms
Wall time: 198 ms


In [20]:
%%time
perform_testing(recognizer, testing_set)

Performing 10 tests

Hosni_Mubarak is correctly recognized with confidence 70.3320301717
Jeong_Se-hyun incorrect (recognized as Jesse_Jackson with confidence 66.9695790301)
Aaron_Peirsol incorrect (recognized as Colin_Farrell with confidence 64.0263118099)
Heizo_Takenaka is correctly recognized with confidence 62.2523363529
Jesse_Jackson incorrect (recognized as Hugh_Grant with confidence 55.9830713166)
Hugh_Grant is correctly recognized with confidence 58.4431455137
Fernando_Gonzalez is correctly recognized with confidence 58.1942826626
George_Clooney incorrect (recognized as Jeong_Se-hyun with confidence 61.8314192222)
Colin_Farrell is correctly recognized with confidence 60.6598527609
Charles_Taylor is correctly recognized with confidence 69.1481984626

Accuracy: 60.00%
Correct:  6
Wrong:    4
CPU times: user 88 ms, sys: 0 ns, total: 88 ms
Wall time: 85.1 ms
